<a href="https://colab.research.google.com/github/Lukas2010/netflix_data/blob/main/netflix_image_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the start of our Colab workbook on image generation using a pythorch backend.

First get the specifics of the allocated hardware:

In [1]:
#@markdown What GPU am I using?

#@markdown V100 > P100 > everything else

!nvidia-smi --query-gpu=gpu_name,gpu_bus_id,vbios_version --format=csv
gpu_name = !nvidia-smi --query-gpu=gpu_name, --format=csv

name, pci.bus_id, vbios_version
Tesla K80, 00000000:00:04.0, 80.21.25.00.02


Connect google Drive:

In [2]:
#@title Connect Google Drive (recommended)
import os
abs_root_path = "/content"

from google.colab import drive
drive.mount('/content/drive')

def ensureProperRootPath():
    if len(abs_root_path) > 0:
        os.chdir(abs_root_path) # Changes directory to absolute root path
        print("Root path check: ")
        !pwd

ensureProperRootPath()

Mounted at /content/drive
Root path check: 
/content


Next you can choose to create a folder in your directoy in you project name folder. 

In [3]:
#@title Make a new folder & set root path to that folder (recommended)
#@markdown Saves a step if you don't have a folder in your Google Drive for this. Makes one, sets the root_path to that new folder. You can name it whatever you'd like:

folder_name = "AI_ART" #@param {type: "string"}
abs_root_path = "/content"
if len(folder_name) > 0:
    path_tmp = abs_root_path + "/drive/MyDrive/" + folder_name
    if not os.path.exists(path_tmp):
        os.mkdir(path_tmp)
    abs_root_path = path_tmp

print("Created folder & set root path to: " + abs_root_path)

#@markdown Make & assign path to a project subfolder (optional)

project_name = "ALL_DATASETS_TEST" #@param {type: "string"}
if len(project_name) > 0:
      path_tmp = abs_root_path + "/" + project_name
      if not os.path.exists(path_tmp):
          os.mkdir(path_tmp)
      abs_root_path = path_tmp
print("Created project subfolder & set root path to: " + abs_root_path)

ensureProperRootPath()

Created folder & set root path to: /content/drive/MyDrive/AI_ART
Created project subfolder & set root path to: /content/drive/MyDrive/AI_ART/ALL_DATASETS_TEST
Root path check: 
/content/drive/MyDrive/AI_ART/ALL_DATASETS_TEST


Install all libraries needed for this project:

In [4]:
# @title Library Installation
import os 
!nvidia-smi
print("Downloading CLIP...")
!git clone https://github.com/openai/CLIP                 &> /dev/null
!git clone https://github.com/crowsonkb/guided-diffusion  &> /dev/null
!pip install -e ./CLIP                                    &> /dev/null
print("Installing library for guided diffusion...")
!pip install -e ./guided-diffusion                        &> /dev/null

print("Installing Python Libraries for AI")
!git clone https://github.com/CompVis/taming-transformers &> /dev/null
!pip install ftfy regex tqdm omegaconf pytorch-lightning  &> /dev/null
!pip install kornia                                       &> /dev/null
!pip install einops                                       &> /dev/null
print("Installing transformers library...")
!pip install transformers                                 &> /dev/null
 
print("Installing libraries for managing metadata...")
!pip install stegano                                      &> /dev/null
!apt install exempi                                       &> /dev/null
!pip install python-xmp-toolkit                           &> /dev/null
!pip install imgtag                                       &> /dev/null
!pip install pillow==7.1.2                                &> /dev/null    
!pip install taming-transformers                          &> /dev/null        
                            

print("Installing ESRGAN for image upscaling...")
!git clone https://github.com/xinntao/ESRGAN &> /dev/null 

print("Installing ffmpeg for creating videos...")
!pip install imageio-ffmpeg &> /dev/null

if not os.path.exists(abs_root_path + "/vqgan-steps"):
    !mkdir "vqgan-steps" &> /dev/null  
    print("No directory for VQGAN+CLIP image output found. Made directory: ~/vqgan-steps")
if not os.path.exists(abs_root_path + "/diffusion-steps"):
    !mkdir "diffusion-steps" &> /dev/null  
    print("No directory for CLIP-guided diffusion image output found. Made directory: ~/diffusion-steps")
!pip freeze > requirements.txt
print("Installation finished.")

Fri Oct  8 13:37:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

List all files in the directory:

In [10]:
os.listdir()

['CLIP',
 'guided-diffusion',
 'taming-transformers',
 'ESRGAN',
 'vqgan-steps',
 'diffusion-steps',
 'requirements.txt',
 'NetflixViewingHistory.csv']

Extraction of the netflix.csv dataset:

In [21]:
import numpy as np
import pandas as pd

print("Import of pandas and np done")

#@title Choose the file you want to upload (Without the ' '): 
file_name = "NetflixViewingHistory.csv" #@param {type: "string"}

netflix_file = pd.read_csv(file_name)

print("File of your choice was imported!")

def extract_titles(netflix_data):
    
    # Add Episode_title column for extracted episode titles

    episode_title = str()

    netflix_data["episode_title"] = episode_title

    for i in range(len(netflix_data)):
    
        sequence = netflix_data.iloc[i,0]
    
        if sequence.count(":") == 2:
        
            series_title = sequence[0:sequence.index(":")]
        
            netflix_data.iloc[i,0] = series_title
        
            episode_title =  sequence[sequence.rfind(":")+2:]
        
            netflix_data.iloc[i,2] = episode_title
        
        elif sequence.count(":") == 3:
        
            first_sequence = sequence[0:sequence.index(":")]

            second_sequence = sequence[len(first_sequence)+1:]

            series_title = second_sequence[1:second_sequence.index(":")]
        
            netflix_data.iloc[i,0] = series_title
        
            episode_title =  sequence[sequence.rfind(":")+2:]
        
            netflix_data.iloc[i,2] = episode_title
        
        else:
        
            netflix_data.iloc[i,2] = "-" 
                
    return netflix_data

Import of pandas and np done


Now save the titles of your movies and series in a file called data and rearrage the column names:

In [87]:
data = extract_titles(netflix_file)  

data = data[['Title','episode_title', 'Date']]

,Title,episode_title,Date
0,The Two Popes,-,28/08/2021
1,Creed,-,10/08/2021
2,Once Upon a Time in Hollywood,-,18/07/2021
3,How to Become a Tyrant,-,16/07/2021
4,Minority Report,-,06/07/2021


This is what your file looks like now:

In [39]:
print(data.head())

                           Title episode_title        Date
0                  The Two Popes             -  28/08/2021
1                          Creed             -  10/08/2021
2  Once Upon a Time in Hollywood             -  18/07/2021
3         How to Become a Tyrant   Seize Power  16/07/2021
4                Minority Report             -  06/07/2021


Select all movies you watched in the last year:

In [105]:
from datetime import datetime, timedelta

today = datetime.today()

one_year_ago = today - timedelta(days = 365)

one_year_ago = one_year_ago.strftime("%d/%m/%Y")

# Convert the Date column to datetime

data.Date = pd.to_datetime(data.Date, format="%d/%m/%Y")

last_year = data[data.Date >= one_year_ago] 

movies_last_year = last_year[last_year.episode_title == "-"]

movies_last_year = movies_last_year[["Title"]].drop_duplicates()

if len(movies_last_year) > 0:

    print("These are your last 5 movies you watched last year:")

    print(movies_last_year.head())

else:
    print("It seems you have watched no movies last year !")

08/10/2020
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
These are your last 5 movies you watched last year:
                           Title
0                  The Two Popes
1                          Creed
2  Once Upon a Time in Hollywood
3         How to Become a Tyrant
4                Minority Report


In [114]:
movies_list = movies_last_year["Title"].to_list()

movies_list[0]

'The Two Popes'

In [124]:
#@title Choose you favourite movie from the menu:

import ipywidgets as widgets

movie_choice = widgets.Dropdown(options = movies_list, value = movies_list[0])

movie_choice

Dropdown(options=('The Two Popes', 'Creed', 'Once Upon a Time in Hollywood', 'How to Become a Tyrant', 'Minori…

In [127]:
#@title You chose:

print(movie_choice.value)

Fauda
